In [3]:
#read and inspect text
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

print(f"length of dataset in characters: {len(text)}")
print(f"first 1000 characters in text:\n {text[:1000]}")

length of dataset in characters: 1115394
first 1000 characters in text:
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the

In [4]:
chars = sorted(list(set(text))) #set of all unique characters that occur in the text
vocab_size = len(chars)
print(f"all unique characters: {''.join(chars)}")
print(f"vocab size: {vocab_size}")

all unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [5]:
#tokenize the text - create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encoder = lambda s: [stoi[c] for c in s] #encoder: take a string, output a list of integers
decoder = lambda l: ''.join([itos[i] for i in l]) #decoder: take a list of integers and output a string

print(f"Encoding 'hello world': {encoder('hello world')}")
print(f"Encoding and Decoding 'hello world': {decoder(encoder('hello world'))}")

Encoding 'hello world': [46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
Encoding and Decoding 'hello world': hello world


In [6]:
#tokenize the entire dataset(input.txt) and store it in a torch.Tensor
import torch
data = torch.tensor(encoder(text), dtype = torch.long)
print(f"shape of data: {data.shape}")
print(f"data type of data: {data.dtype}")
print(f"first 1000 elements of data: {data[:1000]}")

shape of data: torch.Size([1115394])
data type of data: torch.int64
first 1000 elements of data: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49,

In [7]:
#splitting data into train and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context} target it: {target}")

when input is tensor([18]) target it: 47
when input is tensor([18, 47]) target it: 56
when input is tensor([18, 47, 56]) target it: 57
when input is tensor([18, 47, 56, 57]) target it: 58
when input is tensor([18, 47, 56, 57, 58]) target it: 1
when input is tensor([18, 47, 56, 57, 58,  1]) target it: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target it: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target it: 58


In [11]:
torch.manual_seed(1337)
batch_size = 4 #no. of independent sequences to be processed in parallel
block_size = 8 #max. context length for predictions

def get_batch(split):
    #generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i: i+block_size] for i in ix])
    y = torch.stack([data[i+1: i+block_size +1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('INPUTS:')
print(f"Inputs Shape: {xb.shape}")
print(xb)
print("TARGETS")
print(f"Targets Shape: {yb.shape}")
print(yb)
print("---------------------------")
for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = xb[b : t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} target is: {target}")

INPUTS:
Inputs Shape: torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
TARGETS
Targets Shape: torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------------------------
when input is [[24, 43, 58, 5, 57, 1, 46, 43]] target is: 43
when input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58]] target is: 58
when input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58], [52, 58, 1, 58, 46, 39, 58, 1]] target is: 5
when input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58], [52, 58, 1, 58, 46, 39, 58, 1], [25, 17, 27, 10, 0, 21, 1, 54]] target is: 57
when input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58], [52, 58, 1, 58, 46, 39, 58, 1], [

In [20]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        #idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) #(B, T, C) -> batch, time, channel
        if targets == None:
          loss = None
        else:
          B, T, C = logits.shape
          logits = logits.view(B*T, C)
          targets = targets.view(B*T)
          loss = F.cross_entropy(logits, targets)
        return logits, loss #logits -> scores for next character sequence

    def generate(self, idx, max_new_tokens):
        #idx = (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            logits, loss = self(idx) #get the predictions
            logits = logits[:, -1, :] #transforms to (B,C)
            #apply softmax and get probabilities
            probs = F.softmax(logits, dim = -1) #(B,C)
            idx_next = torch.multinomial(probs, num_samples = 1) #(B, 1)
            #append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(f"output shape: {logits.shape}")
print(f"loss: {loss}")
print(decoder(m.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens=100)[0].tolist()))

output shape: torch.Size([32, 65])
loss: 4.878634929656982

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [67]:
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)
for steps in range(100):
    xb, yb = get_batch('train') #sample from text

    #evaluate loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(f"loss: {loss.item()}")

loss: 2.5742409229278564


In [69]:
#generate after training
print(decoder(m.generate(idx = torch.zeros((1,1), dtype = torch.long), max_new_tokens=500)[0].tolist()))


YCx;AUTotha hotirXEav'spJMANzzF-wothPJrisiMk&yZBM,rVgCKpon, IzLMrd d ninthe w emNt f h nLTwe totherowowcHoviPMkOATsskR?EShelow;DUGJmula kno d ingiszol!ocdicy r, CWUk, f d,r b$vqRylyNCY& one bOLI ckt berusonenen&k''d
CjR ton tagdd CG Yan gEs, sa-wossh'hixY33Ku gHAXU,
Wbonxfrirgw, ud IN'T:CKencmathsthitetatiykSVheLIO:
Dnd,be no prd srV&vjbedze woCZlgonoce s viFicFond KIWKun pt ly it mre toig, nkRmyQl, s oualvigXUEHW?Rjyll, EHz:C$
Y,O:wINNy a!UCIDHKkswhee:C on d:
j3s, te tj3both b$phen y IfzPZOERy 
